In [9]:
from importlib import reload

import torch
from torch.utils.data import DataLoader, random_split
from torch.nn import BCELoss
import torch.nn as nn
from torchvision.datasets import CIFAR10
import warnings
warnings.filterwarnings('ignore')

import vitunimodal
import mbt
import ablation
import helpers
reload(mbt)
reload(ablation)
reload(helpers)
reload(vitunimodal)
from helpers import ClassifierMetrics, display_tensor_as_rgb
from vitunimodal import ViTMBT, train, val
from data import EmoDataset, new_collate_fn
from constants import *

In [10]:
DEVICE = "cuda"
PRETRAINED_CHKPT = "./pretrained_models/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224.npz"
EPOCHS = 20
lr = 0.00001
BATCH_SZ = 16
LABELS = 8
vmbt = ViTMBT(1024, num_class=LABELS)
vmbt = nn.DataParallel(vmbt).cuda()

se = None
lines = None
train_ds = EmoDataset(f"{DATA_DIR}/Labels/train_labels_full_pruned.csv", nlines=lines, sole_emotion=se)
val_ds = EmoDataset(f"{DATA_DIR}/Labels/val_labels_full_pruned.csv", nlines=lines, sole_emotion=se)
test_ds = EmoDataset(f"{DATA_DIR}/Labels/test_labels_full_pruned.csv", nlines=lines, sole_emotion=se)
train_dl = DataLoader(train_ds, collate_fn=new_collate_fn, batch_size=BATCH_SZ, shuffle=True)
val_dl = DataLoader(val_ds, collate_fn=new_collate_fn, batch_size=BATCH_SZ, shuffle=False)
test_dl = DataLoader(test_ds, collate_fn=new_collate_fn, batch_size=BATCH_SZ, shuffle=False)

train_metrics = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)
val_metrics = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)


optimizer = torch.optim.AdamW(vmbt.parameters(), betas=(0.9, 0.999), lr=lr, weight_decay=1.0 / BATCH_SZ)
loss_func = BCELoss()
# loss_func = nn.CrossEntropyLoss()

Loading audio layers...
Loaded successfully in 3.8s


In [11]:
for epoch in range(EPOCHS):
    train_metrics = train(vmbt, train_dl, optimizer, loss_fn=loss_func, cls_metrics=train_metrics)
    train_loss, train_f1, train_r, train_p, train_acc = train_metrics
    val_metrics = val(vmbt, val_dl, loss_fn=loss_func, cls_metrics=val_metrics)
    val_loss, val_f1, val_r, val_p, val_acc = val_metrics

    print(
        (f"Epoch {epoch + 1}: train_loss {train_loss:.5f}, val_loss {val_loss:.5f}\n"
            f"                   train_precision {train_p}, val_precision {val_p}\n"
            f"                   train_recall {train_r}, val_recall {val_r}\n"
            f"                   train_f1 {train_f1}, val_f1 {val_f1}"
            )
        )
    

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1: train_loss 0.62026, val_loss 0.62542
                   train_precision 0.4289348607097767, val_precision 0.29988789136420796
                   train_recall 0.16548463356973994, val_recall 0.11486486486486487
                   train_f1 0.19943899012111613, val_f1 0.12217762239313962


  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Epoch 10: train_loss 0.35456, val_loss 0.73298
                   train_precision 0.7286545288403709, val_precision 0.36559500368254777
                   train_recall 0.3877068557919622, val_recall 0.15878378378378377
                   train_f1 0.4350082712927149, val_f1 0.2006503414373113


  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]